In [75]:
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt

In [68]:
path = "/data/share/data/Speech/TMHINTQI/cleaned_test_mos.csv"
data = pd.read_csv(path)
data

,filename,Q,I
0,DDAE_snr-2_babble_TMHINT_b2_21_06,1.000000,0.000000
1,DDAE_snr-2_babble_TMHINT_b2_21_08,1.000000,0.333333
2,DDAE_snr-2_babble_TMHINT_b2_23_04,2.000000,9.333333
3,DDAE_snr-2_babble_TMHINT_b2_25_08,1.000000,1.666667
4,DDAE_snr-2_babble_TMHINT_b2_26_04,2.750000,6.750000
...,...,...,...
1973,clean_TMHINT_g4_32_05,4.666667,10.000000
1974,clean_TMHINT_g4_32_06,5.000000,10.000000
1975,clean_TMHINT_g4_32_07,5.000000,10.000000
1976,clean_TMHINT_g4_32_08,4.000000,10.000000


In [69]:
# rows with clean utterances
uttId2transcript = {}
path = "/data/share/data/Speech/TMHINTQI/transcript_clean_testset_utterances.jsonl"
with open(path, "r", encoding="utf-8") as file:
    rows = [json.loads(line) for line in file]
for row in rows:
    audio_path = row['audio_path']
    uttId = audio_path.split("/")[-1].replace(".wav", "").split("TMHINT_")[-1]
    assert uttId not in  uttId2transcript
    uttId2transcript[uttId] = row['transcript']
print("len(uttId2transcript)", len(uttId2transcript))

len(uttId2transcript) 192


In [76]:
x

filename    clean_TMHINT_g4_32_09
Q                             4.8
I                            10.0
Name: 1977, dtype: object

In [111]:
count = 0
data_pairs = {k: [] for k in uttId2transcript.keys()}
# there are 230 unique 'b' (utterances), of which 192 has 'clean' utterances 
# there are 97 unique systems 'a'
for i in range(len(data)):
    x = data.iloc[i]
    items = x['filename'].split('TMHINT')
    a = items[0].strip("_") # systemId
    b = items[1].strip("_") # uttId

    if b not in data_pairs:
        continue
    wav_path = f"/data/share/data/Speech/TMHINTQI/test/{x['filename']}.wav"
    assert os.path.exists(wav_path)

    data_pairs[b].append(
        {
            'system': a,
            'row_id': i,
            'text': uttId2transcript[b],
            'path': wav_path,
            'Q': x['Q'],
            'I': x['I'],
        }
    )

In [112]:
len(data_pairs)

192

In [113]:
count_pairs = 0
for k, v in data_pairs.items():
    n = len(v)
    count_pairs += int(n * (n-1) * 0.5)
print(count_pairs)

6916


In [120]:
data_pairwise = []
for uttId, data_this_utt in data_pairs.items():
    for i in range(0, len(data_this_utt)-1):
        for j in range(i+1, len(data_this_utt)):
            assert data_this_utt[i]['text'] == data_this_utt[j]['text']
            if data_this_utt[i]['Q'] == data_this_utt[j]['Q']:
                continue
            item = [data_this_utt[i], data_this_utt[j]]
            random.shuffle(item)
            data_pairwise.append(item)

In [125]:
len(data_pairwise)

6475

In [126]:
random.choice(data_pairwise)

[{'system': 'FCN_snr-2_white',
  'row_id': 379,
  'text': '我爸爸最喜欢喝龙井茶',
  'path': '/data/share/data/Speech/TMHINTQI/test/FCN_snr-2_white_TMHINT_g4_21_01.wav',
  'Q': 2.6666666666666665,
  'I': 6.333333333333333},
 {'system': 'FCN_snr-2_street',
  'row_id': 359,
  'text': '我爸爸最喜欢喝龙井茶',
  'path': '/data/share/data/Speech/TMHINTQI/test/FCN_snr-2_street_TMHINT_g4_21_01.wav',
  'Q': 2.0,
  'I': 9.0}]

In [124]:
with open("../data/data_tmhintqi_pairwise_diffall.json", "w") as f:
    json.dump(data_pairwise, f, indent=4, ensure_ascii=False)